In [1]:
# do not upload this cell! 
API_PATH = '/Users/hkromer/01_Projects/10.SolarAnlage/01.Analytics/solarAnalytics/2020-04-16.solarAnalytics/01.Original_data/DarkSky/DarkSkyAPI.pw'
LOCATION_PATH = '/Users/hkromer/01_Projects/10.SolarAnlage/01.Analytics/solarAnalytics/2020-04-16.solarAnalytics/01.Original_data/location_alternate.pw'

# Missing value treatment of DarkSky dataset

In this notebook, we will consider the treatment of the missing values that we identified in the former notebook, which was concerned with the exploratory data analysis, here again the main findings of the analysis: 

- **Overall**
    - There are seven missing days with no data
    
    
- **Temperature**
    - Daytime temperature are mostly above 0 degC, there is a seasonal trend
    - Low daytime temperatures (`apparentTemperatureLow`) are missing from 2018-05-19 until 2018-05-18
    - `apparentTemperatureLow` ranged from -11 degC until 24 degC, mean 15 degC
    - `apparentTemperatureHigh` ranged from -7 degC until 42 degC, mean 5 degC
    - `temperatureHigh` ranged from -5 degC until 37 degC, mean 16 degC
    
    
- **Precipitation**
    - There are missing values in the `precip`-columns for the period from 2018-05-20 to August 2018-05-19
    - There is one outlier in the intensity of precipitation that we corrected for using nearby precipitation values (± 5 days)
    - `precipProbability` ranged from 0 to 1, with mean of 0.45
    - `precipIntensityMax_cm` ranged from 0 cm to 25 cm, with mean of 2 cm per hour
    - `precipTypes` were either rain, snow, none, or missing


- **Cloud coverage**
    - There are missing values (20%) for days before 2018-08-08
    - `cloudCover` ranged from 0 to 1, with mean of 0.6


- **Sun uptime**
    - The hours the sun was up follows a seasonal trend
    - `sun_uptime` ranged between 8 and 16 hours of sun per day, mean of 12 hours
    
    
- **uvIndex**
    - There are missing values in the range where other columns also have missing values
    - There is a big problem with these values, since there was a change in the model that DarkSky uses on August 16, 2018. uvIndizes before that time show a different distribution than how it was after this change. The column might not be as useful in the model
    - `uvIndex` ranged between 0 and 1, mean 3
    
    

## Table of content

## [1. Import libraries](#1) 
## [2. Load dataframe after EDA](#2) 
## [3. Find missing days](#3) 
### [3.1. Fully missing days](#4)
### [3.2. Missing data from May 2018 until August 2018](#5)
## [4. Takeaway](#6) 

<a id=1> </a> 
## 1. Import libraries




In [2]:
import pandas as pd
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import datetime
import pandas as pd
import requests
import re
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
class DarkSkyETL():
    
    def __init__(self, DarkSkyAPI, location, save_path):
        self.API = DarkSkyAPI
        self.LOCATION = location
        self.SAVE_PATH = save_path # where to store output json files. will be stored as save_pathYYYY-MM-DD.json
        
    def call_API(self, date_str, silent=False):
        """
        Calls the DarkSky API for the specified location at the specified date, which must be a string in the
        format YYYY-00-DD. Asks for hourly data and returns a json object for this date.
        Prints the number of hours loaded if silent is True.
        """
        # option list for API
        option_list = "exclude=currently,minutely,hourly,alerts&units=si"
        
        # convert to a timestamp at mid-day
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d') + pd.DateOffset(hours=12)
       
        # convert the date to unit time
        unix_time = int(datetime.datetime.timestamp(date))
        
        # call API
        latitude = LOCATION[0]
        longitude = LOCATION[1]
        api_request = "https://api.darksky.net/forecast/"+DarkSkyAPI+"/"+latitude+longitude+","+str(unix_time)+"?"+option_list
        response = requests.get(api_request)
        json_res = response.json()
#         print(api_request)


        if silent == False:
            print(date)
            # some dates have no data
            if 'daily' in json_res.keys():
                # write to file
                with open(f'{self.SAVE_PATH}/{date_str}.json', 'w') as outfile:  
                    json.dump(json_res, outfile)
                    outfile.close()
#                 print(f"Loaded {datapoints} datapoints for {date_str}")
            else:
                print(f"Daily not in keys for {date_str}")
            
        return json_res
    
    def get_date_range(self, start_date, end_date, silent=False):
        """
        Gets a list of dates in steps of one day between start_date and end_date. Calls the API and saves the json return file at the location save_path.
        INPUT:
            - start_date: STRING, date to start
            - end_date: STRING, date to end the list
            - silent: BOOLEAN, False, if the number of datapoints on each date should be printed out
        OUTPUT:
            None
        """
        
        # convert to a time delta
        d_from_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        d_to_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')
        delta = d_to_date - d_from_date
        
        # for every day in the date range, get the data
        for i in range(delta.days+1):
            date = str((d_from_date + datetime.timedelta(days=i)).date())
            print(date)
            self.call_API(date, silent)

<a id=2> </a> 
## 2. Load dataframe after EDA


In [3]:
df = pd.read_csv('../02.Prepared_data/DarkSky/data_after_EDA.csv', index_col=0, parse_dates=True)
time_cols = ['time', 'sunriseTime', 'sunsetTime']
for c in time_cols:
    df[c] = pd.to_datetime(df[c])
    
df.set_index('time', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 922 entries, 2017-10-01 to 2020-04-16
Data columns (total 11 columns):
apparentTemperatureHigh    922 non-null float64
apparentTemperatureLow     840 non-null float64
cloudCover                 740 non-null float64
precipProbability          837 non-null float64
precipType                 839 non-null object
sunriseTime                922 non-null datetime64[ns]
sunsetTime                 922 non-null datetime64[ns]
temperatureHigh            922 non-null float64
uvIndex                    889 non-null float64
precipIntensityMax_cm      839 non-null float64
sun_uptime                 922 non-null float64
dtypes: datetime64[ns](2), float64(8), object(1)
memory usage: 86.4+ KB


In [4]:
df.describe()

,apparentTemperatureHigh,apparentTemperatureLow,cloudCover,precipProbability,temperatureHigh,uvIndex,precipIntensityMax_cm,sun_uptime
count,922.000000,840.000000,740.000000,837.000000,922.000000,889.000000,839.000000,922.000000
mean,15.049284,4.716512,0.625838,0.452461,15.586985,2.817773,1.588682,11.782646
std,9.064251,6.593031,0.299811,0.383607,8.603653,2.215445,2.531118,2.558343
min,-6.940000,-11.380000,0.000000,0.000000,-4.870000,0.000000,0.000000,8.366667
25%,7.120000,-0.357500,0.400000,0.050000,8.307500,1.000000,0.025908,9.370833
50%,14.160000,4.045000,0.715000,0.420000,14.440000,2.000000,0.448056,11.450000
75%,22.482500,9.342500,0.862500,0.850000,22.762500,4.000000,2.170303,14.079167
max,41.780000,23.870000,1.000000,1.000000,37.490000,9.000000,25.255220,16.050000


<a id=3> </a> 
## 3. Find missing days


<a id=4> </a> 
### 3.1 Fully missing days (from DarkSky not returned)

In [5]:
# let's see if we have data for all the days
start = df.index.min()
end = df.index.max()
day_range = pd.date_range(start=start, end=end, freq='1D')
df.index[0], day_range[0]

(Timestamp('2017-10-01 00:00:00'), Timestamp('2017-10-01 00:00:00', freq='D'))

In [6]:
# what are the missing days
missing_days = [day.date().strftime('%Y-%m-%d') for day in day_range if day not in df.index]
missing_days

['2018-05-21',
 '2018-06-16',
 '2018-07-07',
 '2018-07-08',
 '2018-07-21',
 '2018-07-22',
 '2018-08-11']

These days agree with what we previously (in the EDA step) found. Let's query the DarkSky API for a different location to get this information.

In [7]:
# # use other locations
# for iloc in range(6):
#     LOCATION = [item.strip() for item in open(LOCATION_PATH, 'r').readlines()[iloc].strip().split()]
#     DarkSkyAPI = open(API_PATH, 'r').readlines()[0].strip()
#     for date_str in missing_days:
#         save_path = f'../01.Original_data/DarkSky_alternate_location_{iloc}/'

#         if not os.path.exists(save_path):
#             os.makedirs(save_path)

#         ds = DarkSkyETL(LOCATION, DarkSkyAPI, save_path)

#         a = ds.call_API(date_str, silent=True)

At location 0, all days were found with data! Let's check how that compares to the values from the desired location (where we initially queried). Let us define this as a couple of functions, because we will also use this later

In [8]:
def load_json_into_one_df(DATAFOLDER, cols_to_keep):
    """
    Loads all the json files in the folder DATAFOLDER and returns a pandas dataframe. Will consider only columns in the list cols_to_keep.
    """
    # What json daily files are in the folder

    files = [f'{DATAFOLDER}{item}' for item in os.listdir(DATAFOLDER) if item.endswith('json')]
    n_files = len(files)

    # Dataframe to hold all the DarkSky data
    df_alternate = pd.DataFrame()

    # loop through the files and append to pandas dataframe
    for file in files:
        # load json file
        with open(f'{file}') as f:
            d = json.load(f)
            f.close()
        # append to df if daily in keys
        if 'daily' in d.keys():
            df_alternate = df_alternate.append(d['daily']['data'], sort=False)

    n_imported = len(df_alternate)


    df_alternate = df_alternate.loc[:, cols_to_keep] 
    print(f"Imported {n_imported} datapoints. {n_files-n_imported} had no daily entry.")
    
    return df_alternate


def convert_columns(df):
    """
    Takes the raw dataframe created after json import and converts all columns that contain time do datetime. Also converts the
    precipitation intensity to cm.
    """
    # columns with time information
    cols_time = [c for c in df.columns if ('Time' in c) or ('time' in c)]

    for col in cols_time:
        df[col] = pd.to_datetime(df[col], unit='s').dt.tz_localize('utc').dt.tz_convert('Europe/Zurich')
        
    # make them not timezone aware! careful do this after utc conversion
    for col in cols_time:
        df[col] = df[col].dt.tz_localize(None)
        
    # order the dataframe by time
    df = df.sort_values(by='time')

    # inch to cm
    inch_to_cm = 2.54
    df['precipIntensityMax_cm'] = df['precipIntensityMax'] * inch_to_cm
    # drop the original, we do not need it.
    df = df.drop(columns=['precipIntensityMax'])

    df.set_index('time', inplace=True, drop=True)
    return df

In [9]:
iloc = 0
DATAFOLDER = f'../01.Original_data/DarkSky_alternate_location_{iloc}/'
cols_to_keep = ['time', 'apparentTemperatureHigh',
 'apparentTemperatureLow',
 'cloudCover',
 'precipIntensityMax',
 'precipProbability',
 'precipType',
 'sunriseTime',
 'sunsetTime',
 'temperatureHigh',
 'uvIndex']

df_alternate = load_json_into_one_df(DATAFOLDER, cols_to_keep)
df_alternate = convert_columns(df_alternate)
df_alternate

Imported 7 datapoints. 0 had no daily entry.


/Users/hkromer/anaconda3/envs/solarAnalytics/lib/python3.7/site-packages/pandas/core/indexing.py:1404: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,apparentTemperatureHigh,apparentTemperatureLow,cloudCover,precipProbability,precipType,sunriseTime,sunsetTime,temperatureHigh,uvIndex,precipIntensityMax_cm
time,,,,,,,,,,
2018-05-21,23.63,10.87,0.76,NaN,NaN,2018-05-21 05:44:00,2018-05-21 21:09:00,23.91,5.0,NaN
2018-06-16,26.11,16.05,NaN,NaN,NaN,2018-06-16 05:30:00,2018-06-16 21:32:00,26.39,NaN,NaN
2018-07-07,28.17,14.01,0.44,NaN,NaN,2018-07-07 05:39:00,2018-07-07 21:31:00,28.45,7.0,NaN
2018-07-08,28.04,11.69,NaN,NaN,NaN,2018-07-08 05:40:00,2018-07-08 21:30:00,28.32,8.0,NaN
2018-07-21,24.07,14.95,0.74,NaN,NaN,2018-07-21 05:53:00,2018-07-21 21:20:00,24.35,5.0,NaN
2018-07-22,28.04,13.78,0.55,NaN,NaN,2018-07-22 05:54:00,2018-07-22 21:19:00,28.32,7.0,NaN
2018-08-11,26.18,8.34,NaN,NaN,NaN,2018-08-11 06:19:00,2018-08-11 20:51:00,26.46,5.0,NaN


We see that these days do exist for the nearby location, but no precipitation data was recorded and also missing values in cloud cover and uvIndex. Let's add this value to the dataframe.

In [10]:
df_orig = df.copy()

In [11]:
# append to the original dataframe
df = df_orig.append(df_alternate, sort=False)
df = df.sort_index()

# let's see if we have data for all the days
start = df.index.min()
end = df.index.max()
day_range = pd.date_range(start=start, end=end, freq='1D')
day_range.shape, df.shape

((929,), (929, 11))

Perfect, now we have all days present. Let's check now again the missing values.

In [12]:
# what missing values there are
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing,
                                'absolute_missing': df.isnull().sum()})
missing_value_df

,column_name,percent_missing,absolute_missing
apparentTemperatureHigh,apparentTemperatureHigh,0.000000,0
apparentTemperatureLow,apparentTemperatureLow,8.826695,82
cloudCover,cloudCover,19.913886,185
precipProbability,precipProbability,9.903122,92
precipType,precipType,9.687836,90
sunriseTime,sunriseTime,0.000000,0
sunsetTime,sunsetTime,0.000000,0
temperatureHigh,temperatureHigh,0.000000,0
uvIndex,uvIndex,3.659849,34
precipIntensityMax_cm,precipIntensityMax_cm,9.687836,90


<a id=5> </a> 
### 3.2 Missing data from May 2018 until August 2018

For this period, let us also try to consider the nearby location and try to use data from there. Load it for a long time (10 yrs) to predict other variables in the future.


However, for now we have to put this on hold, because DarkSky has a limit on the requests to the API.

In [13]:
# # use other locations. continue after daily limit
# iloc = 0
# LOCATION = [item.strip() for item in open(LOCATION_PATH, 'r').readlines()[iloc].strip().split()]
# DarkSkyAPI = open(API_PATH, 'r').readlines()[0].strip()
# start_date = '2014-06-09'
# end_date = '2018-12-31'
# save_path = f'../01.Original_data/DarkSky_alternate_location_historical_{iloc}/'
# silent = False

# if not os.path.exists(save_path):
#     os.makedirs(save_path)

# ds = DarkSkyETL(LOCATION, DarkSkyAPI, save_path)

# ds.get_date_range(start_date, end_date, silent)

<a id=6> </a> 
## 4. Takeaway


We managed to find the missing days from the DarkSky dataset. For the other missing values - in the period from May until August 2018 - we have no solution as of yet. Mean imputation does likely not make too much sense because the data is entirely for several days. 

One idea was to pull data from the past 10 years from DarkSky and use that to predict the missing values. Yet, constraints on the call to the API limit this for now. We will hence put that on hold and not bother with it until further data is available.

In [17]:
df.to_csv('../02.Prepared_data/DarkSky/data_after_missing_value_treatment.csv', index=True)